In [1]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2023



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2023.pdf...
🔄 Continuando a partir da página 2201...
📊 Total de páginas no PDF: 2798

🔍 Processando páginas 2201 a 2300...
✅ Bloco 2201-2300 salvo e checkpoint atualizado. (Tempo: 27.95 segundos)

🔍 Processando páginas 2301 a 2400...
✅ Bloco 2301-2400 salvo e checkpoint atualizado. (Tempo: 24.06 segundos)

🔍 Processando páginas 2401 a 2500...
✅ Bloco 2401-2500 salvo e checkpoint atualizado. (Tempo: 21.13 segundos)

🔍 Processando páginas 2501 a 2600...
✅ Bloco 2501-2600 salvo e checkpoint atualizado. (Tempo: 26.70 segundos)

🔍 Processando páginas 2601 a 2700...
✅ Bloco 2601-2700 salvo e checkpoint atualizado. (Tempo: 22.20 segundos)

🔍 Processando páginas 2701 a 2798...
✅ Bloco 2701-2798 salvo e checkpoint atualizado. (Tempo: 23.72 segundos)

🚀 Processamento do ano 2023 concluído! Resultados salvos em 'resultado_2023.csv'.


In [2]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2020



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2020.pdf...
📊 Total de páginas no PDF: 4273

🔍 Processando páginas 1 a 100...
✅ Bloco 1-100 salvo e checkpoint atualizado. (Tempo: 29.66 segundos)

🔍 Processando páginas 101 a 200...
✅ Bloco 101-200 salvo e checkpoint atualizado. (Tempo: 21.28 segundos)

🔍 Processando páginas 201 a 300...
✅ Bloco 201-300 salvo e checkpoint atualizado. (Tempo: 31.16 segundos)

🔍 Processando páginas 301 a 400...
✅ Bloco 301-400 salvo e checkpoint atualizado. (Tempo: 23.60 segundos)

🔍 Processando páginas 401 a 500...
✅ Bloco 401-500 salvo e checkpoint atualizado. (Tempo: 26.23 segundos)

🔍 Processando páginas 501 a 600...
✅ Bloco 501-600 salvo e checkpoint atualizado. (Tempo: 22.69 segundos)

🔍 Processando páginas 601 a 700...
✅ Bloco 601-700 salvo e checkpoint atualizado. (Tempo: 26.59 segundos)

🔍 Processando páginas 701 a 800...
✅ Bloco 701-800 salvo e checkpoint atualizado. (Tempo: 25.58 segundos)

🔍 Processando páginas 801 a 900...
✅ Bloco

KeyboardInterrupt: 

In [ ]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2020



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2020.pdf...
🔄 Continuando a partir da página 1801...
📊 Total de páginas no PDF: 4273

🔍 Processando páginas 1801 a 1900...
✅ Bloco 1801-1900 salvo e checkpoint atualizado. (Tempo: 24.06 segundos)

🔍 Processando páginas 1901 a 2000...
✅ Bloco 1901-2000 salvo e checkpoint atualizado. (Tempo: 25.82 segundos)

🔍 Processando páginas 2001 a 2100...
✅ Bloco 2001-2100 salvo e checkpoint atualizado. (Tempo: 455.29 segundos)

🔍 Processando páginas 2101 a 2200...
✅ Bloco 2101-2200 salvo e checkpoint atualizado. (Tempo: 34.90 segundos)

🔍 Processando páginas 2201 a 2300...
✅ Bloco 2201-2300 salvo e checkpoint atualizado. (Tempo: 26.92 segundos)

🔍 Processando páginas 2301 a 2400...
✅ Bloco 2301-2400 salvo e checkpoint atualizado. (Tempo: 23.85 segundos)

🔍 Processando páginas 2401 a 2500...
✅ Bloco 2401-2500 salvo e checkpoint atualizado. (Tempo: 23.50 segundos)

🔍 Processando páginas 2501 a 2600...
✅ Bloco 2501-2600 salvo e checkpoint a

In [1]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2020



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2020.pdf...
🔄 Continuando a partir da página 4001...
📊 Total de páginas no PDF: 4273

🔍 Processando páginas 4001 a 4100...
✅ Bloco 4001-4100 salvo e checkpoint atualizado. (Tempo: 21.98 segundos)

🔍 Processando páginas 4101 a 4200...
✅ Bloco 4101-4200 salvo e checkpoint atualizado. (Tempo: 22.94 segundos)

🔍 Processando páginas 4201 a 4273...
✅ Bloco 4201-4273 salvo e checkpoint atualizado. (Tempo: 18.82 segundos)

🚀 Processamento do ano 2020 concluído! Resultados salvos em 'resultado_2020.csv'.


In [ ]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2022



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2022.pdf...
📊 Total de páginas no PDF: 2252

🔍 Processando páginas 1 a 100...
✅ Bloco 1-100 salvo e checkpoint atualizado. (Tempo: 20.11 segundos)

🔍 Processando páginas 101 a 200...
✅ Bloco 101-200 salvo e checkpoint atualizado. (Tempo: 17.98 segundos)

🔍 Processando páginas 201 a 300...
✅ Bloco 201-300 salvo e checkpoint atualizado. (Tempo: 20.68 segundos)

🔍 Processando páginas 301 a 400...
✅ Bloco 301-400 salvo e checkpoint atualizado. (Tempo: 19.10 segundos)

🔍 Processando páginas 401 a 500...
✅ Bloco 401-500 salvo e checkpoint atualizado. (Tempo: 19.72 segundos)

🔍 Processando páginas 501 a 600...
✅ Bloco 501-600 salvo e checkpoint atualizado. (Tempo: 19.01 segundos)

🔍 Processando páginas 601 a 700...
✅ Bloco 601-700 salvo e checkpoint atualizado. (Tempo: 17.84 segundos)

🔍 Processando páginas 701 a 800...
✅ Bloco 701-800 salvo e checkpoint atualizado. (Tempo: 19.02 segundos)

🔍 Processando páginas 801 a 900...
✅ Bloco

In [1]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2022



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2022.pdf...
🔄 Continuando a partir da página 2101...
📊 Total de páginas no PDF: 2252

🔍 Processando páginas 2101 a 2200...
✅ Bloco 2101-2200 salvo e checkpoint atualizado. (Tempo: 27.05 segundos)

🔍 Processando páginas 2201 a 2252...
✅ Bloco 2201-2252 salvo e checkpoint atualizado. (Tempo: 12.33 segundos)

🚀 Processamento do ano 2022 concluído! Resultados salvos em 'resultado_2022.csv'.


In [ ]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")

In [1]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2024



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2024.pdf...
📊 Total de páginas no PDF: 2571

🔍 Processando páginas 1 a 100...
✅ Bloco 1-100 salvo e checkpoint atualizado. (Tempo: 19.20 segundos)

🔍 Processando páginas 101 a 200...
✅ Bloco 101-200 salvo e checkpoint atualizado. (Tempo: 18.79 segundos)

🔍 Processando páginas 201 a 300...
✅ Bloco 201-300 salvo e checkpoint atualizado. (Tempo: 20.58 segundos)

🔍 Processando páginas 301 a 400...
✅ Bloco 301-400 salvo e checkpoint atualizado. (Tempo: 20.61 segundos)

🔍 Processando páginas 401 a 500...
✅ Bloco 401-500 salvo e checkpoint atualizado. (Tempo: 20.54 segundos)

🔍 Processando páginas 501 a 600...
✅ Bloco 501-600 salvo e checkpoint atualizado. (Tempo: 19.96 segundos)

🔍 Processando páginas 601 a 700...
✅ Bloco 601-700 salvo e checkpoint atualizado. (Tempo: 20.87 segundos)

🔍 Processando páginas 701 a 800...
✅ Bloco 701-800 salvo e checkpoint atualizado. (Tempo: 18.43 segundos)

🔍 Processando páginas 801 a 900...
✅ Bloco

In [ ]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2021



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2021.pdf...
📊 Total de páginas no PDF: 3755

🔍 Processando páginas 1 a 100...
✅ Bloco 1-100 salvo e checkpoint atualizado. (Tempo: 18.90 segundos)

🔍 Processando páginas 101 a 200...
✅ Bloco 101-200 salvo e checkpoint atualizado. (Tempo: 18.49 segundos)

🔍 Processando páginas 201 a 300...
✅ Bloco 201-300 salvo e checkpoint atualizado. (Tempo: 21.63 segundos)

🔍 Processando páginas 301 a 400...
✅ Bloco 301-400 salvo e checkpoint atualizado. (Tempo: 21.90 segundos)

🔍 Processando páginas 401 a 500...
✅ Bloco 401-500 salvo e checkpoint atualizado. (Tempo: 20.72 segundos)

🔍 Processando páginas 501 a 600...
✅ Bloco 501-600 salvo e checkpoint atualizado. (Tempo: 18.84 segundos)

🔍 Processando páginas 601 a 700...
✅ Bloco 601-700 salvo e checkpoint atualizado. (Tempo: 20.73 segundos)

🔍 Processando páginas 701 a 800...
✅ Bloco 701-800 salvo e checkpoint atualizado. (Tempo: 23.71 segundos)

🔍 Processando páginas 801 a 900...
✅ Bloco

In [1]:
# Instale as bibliotecas necessárias (caso ainda não estejam instaladas)
# !pip install pdfplumber pandas openpyxl

import os
import pandas as pd
import pdfplumber
import re
import time

# =======================
# CONFIGURAÇÕES INICIAIS
# =======================

# Caminho da pasta onde estão os PDFs
base_folder = r'G:\Meu Drive\IPEA\Consulta_autom'

# Nome do arquivo Excel com as políticas
excel_politicas = 'politicas.xlsx'

# Definir tamanho do bloco (quantidade de páginas por vez)
BLOCO_PAGINAS = 100  

# =======================
# LEITURA DO ARQUIVO EXCEL
# =======================

# Ler o Excel garantindo a separação correta das colunas
df_politicas = pd.read_excel(excel_politicas, engine="openpyxl")

# Remover espaços extras dos nomes das colunas
df_politicas.columns = df_politicas.columns.str.strip()

# Garantindo que as colunas obrigatórias existam no Excel
if 'politica' not in df_politicas.columns or 'ano' not in df_politicas.columns:
    raise ValueError("O arquivo politicas.xlsx deve conter as colunas 'politica' e 'ano'.")

# =======================
# SELEÇÃO DO ANO PARA PROCESSAR O PDF
# =======================

ano_escolhido = int(input("\nDigite o ano que deseja analisar (2019-2024): "))

pdf_file = os.path.join(base_folder, f'SIOP_PLOA{ano_escolhido}.pdf')

if not os.path.exists(pdf_file):
    print(f"❌ Arquivo não encontrado: {pdf_file}")
    exit()

print(f"\n📄 Processando PDF {pdf_file}...")

# Criar arquivo CSV específico para salvar resultados deste ano
csv_saida = f'resultado_{ano_escolhido}.csv'
checkpoint_file = f'checkpoint_{ano_escolhido}.txt'  # Arquivo para armazenar a última página processada

# =======================
# VERIFICAR A ÚLTIMA PÁGINA PROCESSADA
# =======================

ultima_pagina_processada = 0
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        try:
            ultima_pagina_processada = int(f.read().strip())  # Lê a última página processada
            print(f"🔄 Continuando a partir da página {ultima_pagina_processada + 1}...")
        except ValueError:
            ultima_pagina_processada = 0  # Se houver erro na leitura, começa do zero

# =======================
# PROCESSAMENTO DO PDF ESCOLHIDO
# =======================

try:
    with pdfplumber.open(pdf_file) as pdf:
        total_paginas = len(pdf.pages)
        print(f"📊 Total de páginas no PDF: {total_paginas}")

        for inicio in range(ultima_pagina_processada, total_paginas, BLOCO_PAGINAS):
            fim = min(inicio + BLOCO_PAGINAS, total_paginas)
            tempo_inicio_bloco = time.time()

            print(f"\n🔍 Processando páginas {inicio + 1} a {fim}...")

            # Extrair texto das páginas do bloco atual
            paginas_texto = []
            for num_pagina in range(inicio, fim):
                try:
                    texto = pdf.pages[num_pagina].extract_text()
                    if texto:
                        paginas_texto.append((num_pagina + 1, ' '.join(texto.split())))
                except Exception as e:
                    print(f"❌ Erro ao processar página {num_pagina + 1}: {e}")

            # Criar lista de resultados para este bloco
            resultados = []
            for _, linha in df_politicas.iterrows():
                politica = linha['politica']
                ano_politica = linha['ano']
                
                # Criar regex para busca da política
                padrao = re.compile(re.escape(politica), re.IGNORECASE)
                
                total_ocorrencias = 0
                paginas_encontradas = []
                
                for num_pagina, texto_pagina in paginas_texto:
                    ocorrencias = len(re.findall(padrao, texto_pagina))
                    if ocorrencias > 0:
                        total_ocorrencias += ocorrencias
                        paginas_encontradas.append(num_pagina)

                # Adicionar resultados ao bloco
                resultados.append({
                    'politica': politica,
                    'ano_politica': ano_politica,
                    f'qtde_{ano_escolhido}': total_ocorrencias,
                    f'paginas_{ano_escolhido}': ','.join(map(str, sorted(set(paginas_encontradas)))) if paginas_encontradas else ''
                })

            # Salvar resultados do bloco no CSV
            df_resultado = pd.DataFrame(resultados)
            df_resultado.to_csv(csv_saida, mode='a', header=not os.path.exists(csv_saida), index=False, encoding='utf-8-sig')

            # **ATUALIZA O CHECKPOINT**
            with open(checkpoint_file, "w") as f:
                f.write(str(fim))  # Salva a última página processada

            tempo_bloco = time.time() - tempo_inicio_bloco
            print(f"✅ Bloco {inicio + 1}-{fim} salvo e checkpoint atualizado. (Tempo: {tempo_bloco:.2f} segundos)")

except Exception as e:
    print(f"❌ Erro ao abrir ou ler o PDF {pdf_file}: {e}")

print(f"\n🚀 Processamento do ano {ano_escolhido} concluído! Resultados salvos em '{csv_saida}'.")


Digite o ano que deseja analisar (2019-2024):  2021



📄 Processando PDF G:\Meu Drive\IPEA\Consulta_autom\SIOP_PLOA2021.pdf...
🔄 Continuando a partir da página 1901...
📊 Total de páginas no PDF: 3755

🔍 Processando páginas 1901 a 2000...
✅ Bloco 1901-2000 salvo e checkpoint atualizado. (Tempo: 25.46 segundos)

🔍 Processando páginas 2001 a 2100...
✅ Bloco 2001-2100 salvo e checkpoint atualizado. (Tempo: 22.52 segundos)

🔍 Processando páginas 2101 a 2200...
✅ Bloco 2101-2200 salvo e checkpoint atualizado. (Tempo: 21.75 segundos)

🔍 Processando páginas 2201 a 2300...
✅ Bloco 2201-2300 salvo e checkpoint atualizado. (Tempo: 25.27 segundos)

🔍 Processando páginas 2301 a 2400...
✅ Bloco 2301-2400 salvo e checkpoint atualizado. (Tempo: 20.01 segundos)

🔍 Processando páginas 2401 a 2500...
✅ Bloco 2401-2500 salvo e checkpoint atualizado. (Tempo: 20.15 segundos)

🔍 Processando páginas 2501 a 2600...
✅ Bloco 2501-2600 salvo e checkpoint atualizado. (Tempo: 24.24 segundos)

🔍 Processando páginas 2601 a 2700...
✅ Bloco 2601-2700 salvo e checkpoint at